In [1]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import numpy as np
import pandas as pd

import re
import json

from tqdm import tqdm

In [2]:
df = pd.read_excel('data_progress/2 PZZ.xlsx')

In [3]:
df.head()

,fio,comment,resolution,part,page,exact_page,orig_id,list_flag
0,Захаренко В. А.,Во все разделы правил землепользования и застр...,"Предложения, относящиеся к предмету публичных ...",1,1,True,0,False
1,Калантарова Ю. В.,На основании вступившего в законную силу судеб...,"Предложения, относящиеся к предмету публичных ...",1,1,True,1,False
2,Ткач Е. В.,"В картах ПЗЗ не отражены ""Защитные зоны ОКН"" (...",Замечание рекомендовано к учёту. Рекомендовать...,1,5,True,2,False
3,Крупенина О. Н.,"На карте Градостроительного зонирования ""Грани...",1. Предложение предусмотрено проектом ПЗЗ. На ...,1,6,True,3,False
4,Терехов А. С.,Внести в Проект правил землепользования и заст...,Предложение не рекомендовано к учёту. Отсутств...,1,6,True,4,False


In [4]:
df.part.value_counts().sort_index()

1     6386
2     3967
3    90524
4      210
5    12924
Name: part, dtype: int64

## 1. Обогащаем простыми фичами

In [5]:
def get_fio_normalized(t):
    l = str(t).replace('.',' ').split()
    try:
        if len(l[0])==1:
            return '{} {}. {}.'.format(l[2].capitalize(), l[0][0].upper(), l[1][0].upper())
        else:
            return '{} {}. {}.'.format(l[0].capitalize(), l[1][0].upper(), l[2][0].upper())
    except:
        return t

In [6]:
df['fio_n'] = df.fio.apply(get_fio_normalized)
fio_n_popularity = df.fio_n.value_counts()
df['fio_n_popularity'] = df.fio_n.apply(str).apply(fio_n_popularity.get)

In [7]:
df["comment"] = df.comment.fillna('')
df["resolution"] = df.resolution.fillna('')

In [8]:
df['comment_len'] = df.comment.apply(str).apply(len)
df['resolution_len'] = df.resolution.apply(str).apply(len)

In [9]:
comments_popularity = df.comment.value_counts()
df['comment_popularity'] = df.comment.apply(comments_popularity.get)
resolution_popularity = df.resolution.value_counts()
df['resolution_popularity'] = df.resolution.apply(resolution_popularity.get)
comments_list_popularity = df.orig_id.value_counts()
df['comment_list_popularity'] = df.orig_id.apply(comments_list_popularity.get)

В ФИО есть коллективные голоса, например, **"Сидорова Н. И. (коллективное + 41 чел.)"**

In [10]:
multiple_patterns = [
    'коллективн..\s*(?:\+|\-|\–)?\s*(?:еще\s*)?(\d+)\s*',
    '(\d+)\s+подпис',
    'количестве (\d+)\s*\(',
    'подпис...? в количестве (\d+)'
]
multiple_regex = re.compile('|'.join(multiple_patterns))

def get_votes_number(t):
    try:
        return int(''.join(multiple_regex.findall(t)[0]))
    except:
        return 1

In [11]:
df["multiplier"] = [get_votes_number(fio) for fio in df.fio]

*Сколько голосов добавилось от коллективных? =)*

In [12]:
df[df.multiplier>1].multiplier.sum() - sum(df.multiplier>1)

14841

## 2. Классифицируем комментарии и резолюции

2 шага:

1. Вручную размечаем по каким-то понятным популярным словам и классификации, и резолюции на 4 класса: за/против/затрудняюсь/неразмеченное
2. Обучаемся и делаем предсказания по всем остальным

### 2.1. Ручная разметка

#### 2.1.1. Ручная разметка комментариев

In [13]:
comment_upvote_signs = ['считаю необходимым проект','отличн','замечательн','хорош','проект понравился', 'нужен порядок в сфере строительства и прозрачность', 'проект хороший','хороший проект','нет возражений','без замечаний','без возражений','нет замечаний','нет предложений','предложений не','замечаний не', 'возражений не', 'проект одобр', 'не возражаю', 'за!','за.','полностью поддерживаем','ознакомились, одобряем','ознакомлены и считаем целесообразным','выступаю за равномерное и гармон','возражений по проекту пзз не имею','надо принять','необходимо принять','проектом землепользования и застройки соглас','ознакомились, одобряем','ознакомлены и считаем целесообразным','возржений не имею','полностью одобряем','отношение положительное','с проектом пзз ознаком','возражений по данному проекту не имею','проект пзз одобряем','проект поддерживаем', 'округа считаем целесообразным', 'вао поддерживаем', 'считаю целесообразным.', 'и считаем необходимым утвердить', 'выражаем ему наше одобрение', 'ознакомлены, согласны', 'поддерживает пзз', "мы поддерживаем проект", "поддерживаем проект правил землепользования и застройки города Москвы", "москвы согласны.", "очень рады потому" ,"поддерживаем проект", "выражаю свою поддержку", "проект благоприятно отразится", "мы поддерживаем этот проект", "очень рады тому", "полностью согласны", ". одобряем", "округа полностью согласны", ", одобряем.", "считаем проект нужным району", "округов согласны", "округа согласны", "принципиально поддерживаем", "голосую за правила", 'я за', 'за принятие', 'за строительство', 'за правила', 'поддержать проект', 'голосую за', 'выражаю поддержку', 'замечаний по проекту не']
comment_downvote_signs = ['нарушени','требую', 'отклонить', 'в отставку', 'не актуал','категорически','просьба в ПЗЗ рассмотреть','есть претензии','прошу исключить','пророшу исключить','прошу отменить публичные слушания', "проект отвергаю", "я против", "прошу в пзз рассмотреть увеличение"]
comment_words_pos = ['согласен','согласна','поддерживаю', 'одобряю', 'за проект', 'за пзз', 'нравится', 'понрави', 'устраивает', 'пойдет', 'пойдёт']
comment_words_neg = ['против ','против.','против,','возражаю']
comment_full_phrase_pos = ['да', 'за', 'одобрено', '---','за','за.','+']
comment_full_phrase_neg = ['не имею','нет','нет.','-']
strong_upvote_signs = ['я поддерживаю правила', 'я поддерживаю проект', 'я поддерживаю пзз',
                      'я выступаю за правила', 'я выступаю за проект', 'я выступаю за пзз',
                      'я за правила', 'я за проект', 'я за пзз']
strong_downvote_signs = ['я против правил', 'я против проекта', 'я против пзз',
                      'я выступаю против правил', 'я выступаю против проекта', 'я выступаю против пзз'
                      'я против правил', 'я против проекта', 'я против пзз']

def get_comment_class(t):
    if t in {'', ' ', 'комментарий не оставлен'} or 'Затрудняюсь ответить' in t:
        return 0
    t = t.lower()+' '
    if t in comment_full_phrase_neg or t.strip() in comment_full_phrase_neg:
        return -1
    if t in comment_full_phrase_pos or t.strip() in comment_full_phrase_pos:
        return 1
    
    for p in strong_upvote_signs:
        if p in t:
            return 1
    for p in strong_downvote_signs:
        if p in t:
            return -1
    for p in comment_downvote_signs:
        if p in t:
            return -1
    for p in comment_upvote_signs:
        if p in t:
            return 1
    for p in comment_words_neg:
        if p in t:
            if ' не '+p in t:
                return 1
            else:
                return -1
    for p in comment_words_pos:
        if p in t:
            if ' не '+p not in t:
                return 1
            else:
                return -1
    if len(t)<30 and ('ознакомил' in t or 'ознакомлен' in t or 'молодцы' in t or 'одобр' in t and 'не одобр' not in t or 'соглас' in t and 'не соглас' not in t):
        return 1
    if re.findall('замечани?й.{0,30} не', t) or re.findall('предложений.{0,30} не', t) or re.findall('претензий.{0,30} не', t):
        return 1
    if len(t)>300:
        return -1
    return None

In [14]:
df['comment_class_manual'] = df.comment.apply(get_comment_class)
df.comment_class_manual.apply(str).value_counts()

1.0     57377
-1.0    45620
0.0      6777
nan      4237
Name: comment_class_manual, dtype: int64

Просмотр неразмеченных комментариев

In [15]:
unmarked_comments = [c for c in comments_popularity.index if get_comment_class(c) is None]

In [16]:
from collections import Counter

def get_top_bigrams(texts):
    return Counter([' '.join(b) for l in [t.lower() for t in texts] for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]).most_common()

def get_top_trigrams(texts):
    return Counter([' '.join(b) for l in [t.lower() for t in texts] for b in zip(l.split(" ")[:-2], l.split(" ")[1:-1], l.split(" ")[2:])]).most_common()

In [17]:
get_top_trigrams(unmarked_comments)[:10]

[('землепользования и застройки', 239),
 ('  ', 228),
 ('правил землепользования и', 207),
 ('по адресу: ул.', 117),
 ('и застройки города', 84),
 ('проект правил землепользования', 71),
 ('проживаю по адресу:', 66),
 ('застройки города москвы', 61),
 ('проживающий по адресу:', 45),
 ('правила землепользования и', 45)]

In [18]:
unmarked_comments[:10]

['Нужно добавить больше ФОКов и велодорожек. Больше спортивных объектов для жителей',
 'Сделайте больше прогулочных зон и парков.',
 'Необходимо больше уделять внимане культурным центрам для детей и пенсионеров.',
 'Необходимо сохранить зеленые насаждения в городе. Прошу учесть мое мнение',
 'При развитии города необходимо сохранить исторический облик города.',
 'Постройте детские кафе.',
 'Стройте больше дорог.',
 'Нужно реконструировать больше школ.',
 'Нужно построить больше дешевых магазинов.',
 'Постройте больше катков с искусственным льдом.']

Проверка самых популярных комментариев

In [19]:
for c in comments_popularity.index[:30]:
    print(comments_popularity[c], get_comment_class(c), '\n\n', c,'\n\n----\n')

10071 1 

 Я поддерживаю проект ПЗЗ, потому что:
1. ПЗЗ на абсолютном большинстве территорий города Москвы закрепляет существующее положение застройки в части сохранения объемов и назначения зданий, строений, сооружений
2. В ПЗЗ отражены те объекты, решения по строительству которых уже приняты.
3. Любое новое строительство становится возможным только после проведения публичных слушаний.
4. ПЗЗ гарантируют строительство всех объектов социальной инфраструктуры, что обеспечивает новое качество жизни людей. В частности, представленная редакция ПЗЗ предусматривает 50 % увеличение объемов застройки объектов здравоохранения.
5. ПЗЗ устанавливает развитие промышленных зон города Москвы преимущественно с сохранением производственной функции, что позволяет сохранить и увеличить рабочие места.
Я за проект ПЗЗ! 

----

4353 1 

 Да, поддерживаю 

----

3708 -1 

 Я против утверждения проекта Правил землепользования и застройки города Москвы в отношении территории Северного административного округа

Проверка самых крупных списочных комментариев

In [20]:
for c in comments_list_popularity.index[:30]:
    comm = df.comment[df.orig_id==c].iloc[0]
    print(comments_list_popularity[c], get_comment_class(comm), '\n\n', comm,'\n\n----\n')

2277 -1 

 Мы. граждане Российской Федерации, жители г. Москвы, требуем отклонитьпредставленный на публичные слушания проект Правил 'землепользования и застройки (далее -проект ПЗЗ) в соответствии с нижеследующим.Проектом ПЗЗ на земельном участке с кадастровым номером 77:08:0009021:1004, по адресному ориентиру ул. Живописная, вл. 21. градостроительный регламент установлен согласно выданному градостроительному плану земельного участка № RU77 212000-016707.Данный земельный участок, согласно Генеральному плану города Москвы,утверждённому Законом города Москвы от 5 мая 2010 г.
№ 17, был включён в границы функциональной специализированной спортивно- рекреационной общественной зоны (№ 59 на карте. Генеральный план города Москвы, книга 2, стр. 43), как она названа в приложении «Параметры планируемого развития функциональных зон» к карге «Функциональные зоны» (рис. 3). Генеральным планом в этой функциональной зоне предусмотрено строительство только объектов физкультуры и спорта (книга 3. стр. 

#### 2.1.2. Ручная разметка резолюций

In [21]:
resolution_upvote_signs = ['принято к свед', 'рекомендовать к уч', 'рекомендовать депа', 'учесть с сохранени', 'рекомендовано к уч', 'рекомендуется к уч', 'замечание рекоменд', 'предложение рекоме', 'рекомендовать учес', 'рекомендовать к', 'прянято к сведению', 'рекомендовано учес', 'принято к учету', 'рекомендуется учес', 'рекомендовано к учету']
resolution_downvote_signs = ['редложения по обсуждаемому проекту отсутствуют','предложение не може', 'предложения, относящ', 'не может ', 'не рек', 'редложения/зам', 'предложения/замеча', 'предложение/замеча', 'в проекте пзз', 'в указанной террит', 'публичных слушаний', 'предложения не мог', 'замечания/предложе', 'проект пзз подгото', 'объекты социальног', 'по адресу ул.судос', 'внесение изменений', 'в проекте пзз учте', 'вопросы отмены гпз', 'вопросы благоустро', 'в проекте пзз указ', 'вопросы администра', 'редложение/замечан', 'предложения относя', 'учет замечания нец', 'учёту. отсутствуют', 'на указанную терри', 'обращение не может', 'вопросы межевания ', 'вопросы образовани', 'учет предложения н', 'отсутствуют обосно', '---\n\nсеменовна', 'в территориальной ', '---\n\nниколаевна', 'замечания не могут', 'подготовка докумен', 'на карте градостро', 'отсутствует здание', 'высказаны устно пр', ', относящиеся к пр', 'не подлежит учету.', 'замечания и предло', 'замечания, относящ', 'предложение предус', 'предложения и заме', 'публичные слушания', 'зоны, в которой расположены указанные', 'даны разъяснения в ходе']
full_len_downvote_signs = ['замечание не рекомендовано', 'к предмету публичных', 'редложения по обсуждаемому проекту отсутствуют', 'е содержит предложений и замечаний', 'редложение не рекомендовано к учету', 'е рекомендовано к', 'проекту пзз, отсутствуют', 'проекту ПЗЗ, отсутствуют']

def get_resolution_class(t):
    for p in full_len_downvote_signs:
        if p in t:
            return -1
    if not t:
        return None
    t = t.lower().replace('  ',' ')[:50]
    for p in resolution_upvote_signs:
        if p in t:
            return 1
    for p in resolution_downvote_signs:
        if p in t:
            return -1
    if re.findall('предложени[е|я] рек', t):
        return 1
    return None

In [22]:
df['resolution_class_manual'] = df.resolution.apply(get_resolution_class)

In [23]:
df.resolution_class_manual.apply(str).value_counts()

1.0     60777
-1.0    49060
nan      4174
Name: resolution_class_manual, dtype: int64

In [24]:
unmarked_resolutions = [(resolution_popularity[c], c) for c in resolution_popularity.index if get_resolution_class(c) is None]

In [25]:
get_top_trigrams([i[1] for i in unmarked_resolutions])[20:][:10]

[(' с ', 43),
 ('  объектов', 42),
 ('соответствии с положениями', 41),
 ('и  ', 41),
 ('перечень видов разрешенного', 39),
 ('на  ', 38),
 ('  на', 38),
 ('в установленном порядке.', 38),
 ('земельных участков и', 38),
 (' объектов ', 35)]

In [26]:
unmarked_resolutions

[(1332,
  'распространяется на земельные участки, расположенные в границах\xa0территорий памятников и ансамблей, включенных в единый государственный\xa0реестр объектов культурного наследия (памятников истории и культуры)\xa0народов Российской Федерации, а также в границах территорий памятников\xa0или ансамблей, которые являются выявленными объектами культурного\xa0наследия и решения о режиме содержания, параметрах реставрации, консервации, воссоздания, ремонта и приспособлении которых принимаются в порядке, установленном законодательством Российской Федерации об охране объектов культурного наследия.'),
 (1032, ''),
 (162,
  'специального назначения, зоны размещения военных объектов и иные виды территориальных зон. Таким образом, федеральное законодательство допускает возможность введения иных, прямо не перечисленных в ч.1 ст.35 Градостроительного кодекса РФ, видов территориальных зон.'),
 (153, 'Москвы.'),
 (142,
  'действующему ГПЗУ №RU77-195000-004568 выданного в соответствии с\xa0ре

In [27]:
for r in resolution_popularity.index[:30]:
    print(resolution_popularity[r], get_resolution_class(r), '\n\n', r,'\n\n----\n')

34515 1 

 Принято к сведению 

----

22554 1 

 Принято к сведению. 

----

6332 -1 

 Замечание не рекомендовано к учету. В проекте ПЗЗ для территориальной зоны, в которой расположены указанные земельные участки, установлены градостроительные регламенты, соответствующие проекту планировки территории транспортно-пересадочного узла "Ленинградское шоссе", прошедшему публичные слушания в установленном порядке. 

----

2704 -1 

 Замечание не рекомендовано к учету. В проекте ПЗЗ для территориальной зоны, в которой расположены указанные земельные участки, установлены градостроительные регламенты, соответствующие действующему проекту планировки территории, ограниченной улицей Золоторожский Вал, проездом завода Серп и Молот, шоссе Энтузиастов, утвержденному постановлением Правительства Москвы от 03.11.2015 № 723-ПП «О проекте планировки территории района Лефортово г. Москвы» 

----

2288 -1 

 Замечание не рекомендовано к учету. В проекте ПЗЗ для территориальной зоны, в которой расположен ук

### 2.2. Классификация на основе tfidf

#### 2.2.1. Нормализуем все слова

In [28]:
from string import punctuation
import pymorphy2

In [29]:
class LemmatizeText:

    def __init__(self, morph_analyzer_class=pymorphy2.MorphAnalyzer):
        self.morph_analyzer = morph_analyzer_class()
        self.normalization_cache = dict()

    def __call__(self, text):
        text_lowered = text.lower()
        normalized_words_list = []
        for word in text_lowered.split():
            if "_" in word:
                normalized_words_list.append(word)
            else:
                cached_word = self.normalization_cache.get(word, None)
                if not cached_word:
                    word_normalized = self.morph_analyzer.normal_forms(word)[0]
                    if word_normalized.endswith(u"ся"):  # убираем частицу 'ся' у возвратных глаголов
                        word_normalized = word_normalized[:-2]
                    cached_word = self.normalization_cache[word] = word_normalized
                normalized_words_list.append(cached_word)
        normalized_text = " ".join(normalized_words_list)
        return normalized_text

In [30]:
class SeparatePunctuation:
    def __init__(self):
        self.regex = re.compile('([{}])'.format(punctuation))
    
    def __call__(self, text):
        for p in punctuation:
            text = text.replace(p, ' '+p+' ')
        return text

In [31]:
class Tokenizer:
    def __init__(self):
        self.regex_num = re.compile('\d+')
    
    def __call__(self, text):
        return self.regex_num.sub(' num ', text)

In [32]:
class Normalizer:
    def __init__(self):
        self.spec = [SeparatePunctuation(), Tokenizer(), LemmatizeText()]
    
    def __call__(self, text):
        for f in self.spec:
            text = f(text)
        return text

In [33]:
normalizer = Normalizer()

In [34]:
comment_normalized = [normalizer(c) for c in tqdm(df.comment)]

100%|██████████| 114011/114011 [00:39<00:00, 2867.99it/s] 


In [35]:
resolution_normalized = [normalizer(r) for r in tqdm(df.resolution)]

100%|██████████| 114011/114011 [00:09<00:00, 11404.19it/s]


#### 2.2.2. Варим фичи

Векторизуем предложения

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
from string import punctuation
regex_extra_spaces = re.compile('\s+')
regex_punctuation = re.compile('[%s]' % re.escape(punctuation))

def remove_punctuation_and_extra_spaces(t):
    t = str(t)
    t = regex_punctuation.sub(' ', t)
    t = regex_extra_spaces.sub(' ', t)
    t = t.strip()
    return t

def get_first_words(t, words_num=40):
    return " ".join(remove_punctuation_and_extra_spaces(t).split()[:words_num])

In [38]:
vectorizer = TfidfVectorizer(max_features=500, min_df=0.05, max_df=0.95)
comment_vectorized = vectorizer.fit_transform([get_first_words(c) for c in tqdm(comment_normalized)])
Xcomments = pd.DataFrame(comment_vectorized.toarray(), columns=['c{}'.format(i) for i in range(comment_vectorized.shape[1])])

100%|██████████| 114011/114011 [00:23<00:00, 4816.43it/s]


In [39]:
vectorizer_r = TfidfVectorizer(max_features=500, min_df=0.05, max_df=0.95)
resolution_vectorized = vectorizer_r.fit_transform([get_first_words(r) for r in tqdm(resolution_normalized)])
Xresolutions = pd.DataFrame(resolution_vectorized.toarray(), columns=['r{}'.format(i) for i in range(resolution_vectorized.shape[1])])

100%|██████████| 114011/114011 [00:05<00:00, 19895.31it/s]


Простые фичи

In [40]:
Xfeatures = pd.DataFrame(
    {
        'comment_len': [len(c) for c in comment_normalized],
        'resolution_len': [len(r) for r in resolution_normalized],
        'signs_num': [len(re.findall('!', c)) for c in comment_normalized],
        'points_num': [len(re.findall('\.', c)) for c in comment_normalized],
        'comma_num': [len(re.findall(',', c)) for c in comment_normalized],
        'list_items_num': [len(re.findall('\d+\.', r)) for r in df.resolution],
        'caps_words_number': [len(re.findall('[А-Я][А-Я]+', c)) for c in df.comment]
    }
)

Составляем итоговую матрицу для обучения

In [41]:
Xfull = pd.concat([Xfeatures, Xcomments, Xresolutions], axis=1)
Xfull.shape

(114011, 162)

Делаем обучающую выборку, вычёркивая не проставленные метки

In [42]:
marked = (~df.comment_class_manual.isnull())&(~df.resolution_class_manual.isnull())
print(marked.sum())

X = Xfull[marked]
y_comment = pd.DataFrame((df.comment_class_manual[marked]>0)*1)
y_resolution = pd.DataFrame((df.resolution_class_manual[marked]>0)*1)

X.index = range(len(X))
y_comment.index = range(len(X))
y_resolution.index = range(len(X))

105746


#### 2.2.3. Обучаемся

In [43]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
clf_c_rf = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_split=5, random_state=0)
clf_r_rf = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_split=5, random_state=0)

In [45]:
%%time
clf_c_rf.fit(X, y_comment)

CPU times: user 27.4 s, sys: 1.06 s, total: 28.4 s
Wall time: 30.5 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [46]:
%%time
clf_r_rf.fit(X, y_resolution)

CPU times: user 23.5 s, sys: 483 ms, total: 24 s
Wall time: 24.5 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

#### 2.2.4. Предсказываем

In [47]:
%%time
comment_prediction = clf_c_rf.predict(Xfull)

CPU times: user 3.3 s, sys: 990 ms, total: 4.29 s
Wall time: 4.67 s


In [48]:
%%time
resolution_prediction = clf_r_rf.predict(Xfull)

CPU times: user 2.56 s, sys: 400 ms, total: 2.96 s
Wall time: 2.99 s


In [49]:
df["comment_class_prediction"] = ((comment_prediction-0.5)*2).astype(int)
df["resolution_class_prediction"] = ((resolution_prediction-0.5)*2).astype(int)

Как получилось?

In [50]:
df.comment_class_prediction.apply(str).value_counts()

1     60293
-1    53718
Name: comment_class_prediction, dtype: int64

In [51]:
df.groupby('comment_class_manual').comment_class_prediction.value_counts()

comment_class_manual  comment_class_prediction
-1.0                  -1                          45111
                       1                            509
 0.0                  -1                           5561
                       1                           1216
 1.0                   1                          56461
                      -1                            916
Name: comment_class_prediction, dtype: int64

In [52]:
df[df.comment_class_manual.isnull()].comment_class_prediction.value_counts()

-1    2130
 1    2107
Name: comment_class_prediction, dtype: int64

In [53]:
df[df.comment_class_manual.isnull()][['comment','comment_class_prediction']]

,comment,comment_class_prediction
5,Реализация действующего постановления Правител...,-1
6,Представить правила застройки и землепользован...,-1
13,Целевое назначение постройки Рабочая 35 «Дом б...,-1
14,Предложения и замечания дополнительно пришлю п...,-1
16,Прошу перевести из зоны сохранения в зону разв...,-1
17,Прошу внести технико-экономические показатели ...,-1
20,Цель программы понятна.,1
21,Цель программы понятна.,1
24,Ознакомлена с правилами землепользования и зас...,-1
30,От имени своих избирателей выражаю возмущение ...,-1


Смешаем два столбца классификации в один новый: все не ручные заполним из предсказаний

In [54]:
df["comment_class"] = [df.comment_class_manual[i] if not np.isnan(df.comment_class_manual[i]) else df.comment_class_prediction[i] for i in range(len(df))]

In [55]:
df["resolution_class"] = [df.resolution_class_manual[i] if not np.isnan(df.resolution_class_manual[i]) else df.resolution_class_prediction[i] for i in range(len(df))]

## 4. Сохраняем результат

In [56]:
df.head()

,fio,comment,resolution,part,page,exact_page,orig_id,list_flag,fio_n,fio_n_popularity,...,comment_popularity,resolution_popularity,comment_list_popularity,multiplier,comment_class_manual,resolution_class_manual,comment_class_prediction,resolution_class_prediction,comment_class,resolution_class
0,Захаренко В. А.,Во все разделы правил землепользования и застр...,"Предложения, относящиеся к предмету публичных ...",1,1,True,0,False,Захаренко В. А.,2.0,...,1,3,1,1,-1.0,-1.0,-1,-1,-1.0,-1.0
1,Калантарова Ю. В.,На основании вступившего в законную силу судеб...,"Предложения, относящиеся к предмету публичных ...",1,1,True,1,False,Калантарова Ю. В.,3.0,...,1,1,1,1,-1.0,-1.0,-1,-1,-1.0,-1.0
2,Ткач Е. В.,"В картах ПЗЗ не отражены ""Защитные зоны ОКН"" (...",Замечание рекомендовано к учёту. Рекомендовать...,1,5,True,2,False,Ткач Е. В.,4.0,...,1,1,1,1,-1.0,-1.0,-1,-1,-1.0,-1.0
3,Крупенина О. Н.,"На карте Градостроительного зонирования ""Грани...",1. Предложение предусмотрено проектом ПЗЗ. На ...,1,6,True,3,False,Крупенина О. Н.,1.0,...,1,1,1,1,1.0,-1.0,-1,-1,1.0,-1.0
4,Терехов А. С.,Внести в Проект правил землепользования и заст...,Предложение не рекомендовано к учёту. Отсутств...,1,6,True,4,False,Терехов А. С.,2.0,...,1,446,1,1,-1.0,-1.0,-1,-1,-1.0,-1.0


In [57]:
df.columns

Index(['fio', 'comment', 'resolution', 'part', 'page', 'exact_page', 'orig_id',
       'list_flag', 'fio_n', 'fio_n_popularity', 'comment_len',
       'resolution_len', 'comment_popularity', 'resolution_popularity',
       'comment_list_popularity', 'multiplier', 'comment_class_manual',
       'resolution_class_manual', 'comment_class_prediction',
       'resolution_class_prediction', 'comment_class', 'resolution_class'],
      dtype='object')

In [58]:
df.to_excel('data_progress/3 PZZ.xlsx')

------